In [1]:
pipelines


{'__default__': Pipeline([
Node(compute_m3_regul, 'reflex_m3_wide', 'reflex_m3_regul', 'compute_m3_regul'),
Node(standardize_m3, 'm3_stock_dataset', 'm3_stock_parquet', 'standardize_m3'),
Node(standardize_reflex, 'reflex_stock_dataset', 'reflex_stock_parquet', 'standardize_reflex'),
Node(map_m3, {'m3_df': 'm3_stock_parquet', 'rules': 'params:m3_mapping_rules', 'pos_df': 'm3_po_dataset'}, 'm3_map', 'map_m3'),
Node(map_reflex, {'reflex_df': 'reflex_stock_parquet', 'mapping': 'params:reflex_mapping_rules'}, 'reflex_map', 'map_reflex_category'),
Node(build_reflex_m3_wide_with_lotless, {'reflex_cat': 'reflex_map', 'm3_cat': 'm3_map', 'depots': 'params:m3_depots_columns'}, 'corr_dataset', 'build_reflex_m3_wide'),
Node(compute_m3_reliquat, {'m3_cat': 'm3_map', 'reflex_cat': 'reflex_map'}, 'm3_reliquat', 'compute_m3_reliquat_node'),
Node(generate_api_m3_rfx, {'reflex_m3_regul': 'reflex_m3_regul', 'm3_cat': 'm3_map'}, 'stock_m3_rfx', 'generate_stock_m3_rfx_node')
]), 'preprocessing': Pipeline(

In [2]:
catalog


'm3_stock_dataset': kedro_datasets.pandas.sql_dataset.SQLQueryDataset
'm3_po_dataset': kedro_datasets.pandas.sql_dataset.SQLQueryDataset
'reflex_stock_dataset': kedro_datasets.pandas.sql_dataset.SQLQueryDataset
'm3_stock_parquet': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'reflex_stock_parquet': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'm3_map': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'reflex_map': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'm3_reliquat': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'corr_dataset': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'reflex_m3_regul': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'stock_m3_rfx': kedro_datasets.pandas.csv_dataset.CSVDataset
'parameters': kedro.io.memory_dataset.MemoryDataset(data='<dict>')
'params:reflex_mapping_rules': kedro.io.memory_dataset.MemoryDataset(data='<dict>')
'params:reflex_mapping_rules.STD': kedro.io.memory_dataset.MemoryDataset(data='<str>')


## Analyse des cas 

### Cas listés 
https://wolfsa.sharepoint.com/sites/ERP/Documents%20partages/General/20%20Implementation/40%20Go%20live/Stocks-WMS/STOCK_251208/Base%20de%20travail%20stocks%20RFX%20vs%20M3%20NEW_251108.xlsx?web=1

In [12]:
m3 = catalog.load('m3_cat_web_sms')

[12/15/25 16:08:40] INFO     Loading data from m3_cat_web_sms (ParquetDataset)...              ]8;id=625434;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=59156;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#1048\1048]8;;\

In [13]:
m3

,sku,sku_m3,lot,depot,emplacement,type,qty_m3,category,is_150,is_sms
0,024530110040,024530110040,1000002136,200,DES,A01,42.0,DES,0,0
1,024530110040,024530110040,1000002266,200,DES,A01,601.0,DES,0,0
2,024530110042,024530110042,1000002285,200,DES,A01,301.0,DES,0,0
3,024530110044,024530110044,1000002285,200,DES,A01,281.0,DES,0,0
4,024530110046,024530110046,1000002285,200,DES,A01,89.0,DES,0,0
...,...,...,...,...,...,...,...,...,...,...
28701,SCDIVERS004D,STO000000000019,None,100,STOCK,B01,9.0,STOCK,0,0
28702,SCDIVERS005B,STO000000000021,None,100,STOCK,B01,4.0,STOCK,0,0
28703,SCDIVERS0059,STO000000000025,None,100,STOCK,B01,5.0,STOCK,0,0
28704,SCDIVERS0058,STO000000000026,None,100,STOCK,B01,9.0,STOCK,0,0


In [14]:
m3_groups = m3.groupby(["depot","lot", "emplacement", "type", "sku"], dropna=False)["qty_m3"].sum().reset_index()

In [15]:
m3_pivot = m3_groups.pivot_table(
            index=["lot", "type", "sku", "emplacement"],
            columns=["depot"],
            values="qty_m3",
            aggfunc="sum",
            fill_value=0,
        ).reset_index()

In [ ]:
m3_pivot


In [17]:
lotless_m3 = m3[m3['lot'].isnull()]

In [18]:
lotless_m3

,sku,sku_m3,lot,depot,emplacement,type,qty_m3,category,is_150,is_sms
11,02XAO34GCA00TUX,02XAO34GCA00TUX,None,100,STOCK,B01,206.0,STOCK,0,0
12,02XAO34GCA00TUX,02XAO34GCA00TUX,None,150,STOCK,B01,206.0,STOCK,0,0
186,02XAO34HCY00TUX,02XAO34HCY00TUX,None,100,STOCK,B01,128.0,STOCK,0,0
187,02XAO34HCY00TUX,02XAO34HCY00TUX,None,150,STOCK,B01,3.0,STOCK,0,0
22900,85XAO96JQZ00TUX,85XAO96JQZ00TUX,None,100,STOCK,B01,6104.0,STOCK,0,0
...,...,...,...,...,...,...,...,...,...,...
28701,SCDIVERS004D,STO000000000019,None,100,STOCK,B01,9.0,STOCK,0,0
28702,SCDIVERS005B,STO000000000021,None,100,STOCK,B01,4.0,STOCK,0,0
28703,SCDIVERS0059,STO000000000025,None,100,STOCK,B01,5.0,STOCK,0,0
28704,SCDIVERS0058,STO000000000026,None,100,STOCK,B01,9.0,STOCK,0,0
